In [ ]:
#libraries
#We had to downgrade our Python environment to get Tensorflow to run (which was needed for Lobe)
#Lobe install instructions: https://github.com/lobe/lobe-python
from lobe import ImageModel

#OpenCV install instructions: https://www.geeksforgeeks.org/how-to-install-opencv-for-python-in-windows/ (or install via Anaconda)
import cv2

print('There are 2 models to choose from:')
print('1. Model with 92% Lobe accuracy over 6980 pictures')
print('2. Model with 88% Lobe accuracy over 9422 pictures')
choice = int(input('Type the number of your choice: '))

if choice == 1:
    model = ImageModel.load('<path to file>/Cards 6980 92 Tensor Flow/Cards 2 PrePlay Backup TensorFlow/')
else:
    model = ImageModel.load('<path to file>/Cards 9422 88 Tensor Flow/Cards 2 TensorFlow/')

print('Model ' + str(choice) + ' loaded')

In [ ]:
#The next few lines are not used as part of this program (other than the abilityDict), but they would be used if this game were implemented more fully as part of playing with others:
#Also, all difficulty checks are set to a difficulty of 7, and the difficulty is also shown to the player. If this were used not as just a tech demo, the difficulty would normally not be shown, and the difficulty number would change.
knowledgeDict = {'Clubs':{'Arcana':3,'Art':3,'Craft':5,'Culture':1,'Enginery':5,'Finance':1,'History':1,'Medicine':3,'Politics':1,'Ranger':1,'Religion':1,'Sciences':5},'Diamonds':{'Arcana':0,'Art':5,'Craft':3,'Culture':5,'Enginery':0,'Finance':0,'History':0,'Medicine':0,'Politics':1,'Ranger':3,'Religion':1,'Sciences':0},'Hearts':{'Arcana':0,'Art':0,'Craft':0,'Culture':0,'Enginery':3,'Finance':0,'History':0,'Medicine':5,'Politics':0,'Ranger':5,'Religion':0,'Sciences':0},'Spades':{'Arcana':0,'Art':0,'Craft':3,'Culture':0,'Enginery':3,'Finance':0,'History':0,'Medicine':5,'Politics':0,'Ranger':9,'Religion':0,'Sciences':0}}
socialDict = {'Clubs':{'Charm':0,'Courtesy':0,'Fellowship':0,'Intimidate':0,'Plead':3,'Reassure':0},'Diamonds':{'Charm':5,'Courtesy':5,'Fellowship':5,'Intimidate':0,'Plead':5,'Reassure':5},'Hearts':{'Charm':3,'Courtesy':3,'Fellowship':3,'Intimidate':1,'Plead':1,'Reassure':3},'Spades':{'Charm':0,'Courtesy':0,'Fellowship':3,'Intimidate':9,'Plead':0,'Reassure':1}}
physicalDict = {'Clubs':{'Combat':1,'Haul':1,'Traverse':1},'Diamonds':{'Combat':1,'Haul':1,'Traverse':1},'Hearts':{'Combat':9,'Haul':3,'Traverse':9},'Spades':{'Combat':9,'Haul':9,'Traverse':5}}
abilityDict = {'Assess the Mind':{'Suit':'Diamonds','Clubs':0,'Diamonds':1,'Hearts':1,'Spades':0}, 'Astonishing Dodge':{'Suit':'Spades','Clubs':1,'Diamonds':0,'Hearts':0,'Spades':1}, 'Bluff':{'Suit':'Hearts','Clubs':0,'Diamonds':1,'Hearts':1,'Spades':0},'Brilliant':{'Suit':'Diamonds','Clubs':0,'Diamonds':1,'Hearts':0,'Spades':0}, 'Brutal':{'Suit':'Clubs','Clubs':1,'Diamonds':0,'Hearts':0,'Spades':0}, 'Burst of Smoke':{'Suit':'Clubs','Clubs':1,'Diamonds':1,'Hearts':0,'Spades':0}, 'Cold Read':{'Suit':'Hearts','Clubs':0,'Diamonds':0,'Hearts':1,'Spades':0}, 'Excellent Finesse':{'Suit':'Spades','Clubs':0,'Diamonds':0,'Hearts':1,'Spades':1}, 'Insightful Recovery':{'Suit':'Diamonds','Clubs':1,'Diamonds':1,'Hearts':0,'Spades':0}, 'Intimidating Presence':{'Suit':'Hearts','Clubs':1,'Diamonds':0,'Hearts':1,'Spades':0}, 'King of Preparation':{'Suit':'Diamonds','Clubs':0,'Diamonds':1,'Hearts':0,'Spades':1}, 'Knock-out King':{'Suit':'Clubs','Clubs':1,'Diamonds':0,'Hearts':0,'Spades':1}, 'Lure':{'Suit':'Hearts','Clubs':0,'Diamonds':0,'Hearts':1,'Spades':1}, 'Silent King':{'Suit':'Spades','Clubs':0,'Diamonds':0,'Hearts':0,'Spades':1}, 'Strong Recovery':{'Suit':'Clubs','Clubs':1,'Diamonds':0,'Hearts':1,'Spades':0}, 'Vanish':{'Suit':'Spades','Clubs':0,'Diamonds':1,'Hearts':0,'Spades':1}}
stealth = 0
combat = 0
charm = 0
intellect = 0

#Function that takes image and runs it by ML model, returning the number in one variable, suit in the other
#This should ask the user if it predicted the card correctly
def modelPredict(imageFile):
    result = model.predict_from_file(imageFile)
   
    print('')
    print("We think your card was:")
    for label, confidence in result.labels:
        print(f"{label} with confidence of {confidence*100}%")
        correctChoice = str(input("Is that correct?(y or n)"))
        if correctChoice == "y":
            return (label)
        
#Function that takes the character suit, the action, the difficulty and returns true, false, or invalid card
#It also calls modelPredict function
def abilityCheck(charSuit,ability,difficulty,cardChoice):
    rescanCard = 1
    print("The difficulty of the check is " + str(difficulty))
    while rescanCard == 1:
        card = modelPredict(cardChoice)
        spaceLoc = card.find(" ")
        cardNum = card[0:spaceLoc]
        cardSuit = card[spaceLoc+1:]
        
        if cardNum == "J":
            cardNum = 11
        elif cardNum == "Q":
            cardNum = 12
        elif cardNum == "K":
            cardNum = 13
        elif cardNum == "A":
            cardNum = 14
        else:
            cardNum = int(cardNum)

        if abilityDict[ability]['Suit'] != cardSuit:
            print("Rescan card, the one you scanned is not associated with the selected ability, need a card with suit " + abilityDict[ability]['Suit'])
            return -1
        else:
            rescanCard = 0            
            if (cardNum < difficulty):
                return 0
            else:
                return 1

# sn beg - changed imgSel() to use the webCam
# original imgSel is in the if not cap.isOpened() block other than the first line, and moved setting of photopath earlier in func
# Code adapted from "OpenCV with Python By Example" by Prateek Joshi
# https://subscription.packtpub.com/book/application_development/9781785283932/3/ch03lvl1sec28/accessing-the-webcam

def imgSel():
    photoPath = '<path to file>/TestImages/'
    # create video capture object: cap
    cap = cv2.VideoCapture(0)

    # Check if the webcam is opened correctly, and if not use local set of card images
    if not cap.isOpened():
        #raise IOError("Cannot open webcam"), instead we will tell them we couldn't get the webcam working.
        print("Looks like your webcam isn't working. Let's use the default card images for this game.")
        print("Choose one of the image cards below to play:")
        print("1. 6 of Hearts")
        print("2. 6 of Diamonds")
        print("3. 6 of Spades")
        print("4. 6 of Clubs")
        print("5. 10 of Hearts")
        print("6. 10 of Diamonds")
        print("7. 10 of Spades")
        print("8. 10 of Clubs")
        print("9. Jack of Hearts")
        print("10. Ace of Diamonds")
        print("11. King of Spades")
        print("12. Queen of Clubs")
        cardChoice = int(input("Type the number of your choice: "))
    
        if cardChoice == 1:
            return(photoPath+'6Hearts.jpg')
        elif cardChoice == 2:
            return(photoPath+'6Diamonds.jpg')
        elif cardChoice == 3:
            return(photoPath+'6Spades.jpg')
        elif cardChoice == 4:
            return(photoPath+'6Clubs.jpg')
        elif cardChoice == 5:
            return(photoPath+'10Hearts.jpg')
        elif cardChoice == 6:
            return(photoPath+'10Diamonds.jpg')
        elif cardChoice == 7:
            return(photoPath+'10Spades.jpg')
        elif cardChoice == 8:
            return(photoPath+'10Clubs.jpg')
        elif cardChoice == 9:
            return(photoPath+'JackHearts.jpg')
        elif cardChoice == 10:
            return(photoPath+'AceDiamonds.jpg')
        elif cardChoice == 11:
            return(photoPath+'KingSpades.jpg')
        else:
            return(photoPath+'QueenClubs.jpg')
    
    print("Hit the space bar to take a picture of your card: ")
    #Loop keep reading webCam frames until there's a keyboard interrupt
    while True:
        ret, frame = cap.read()
        frame = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
        cv2.imshow('Input', frame)

        c = cv2.waitKey(1) #32 is ascii space, stop reading frames when they hit it
        if c == 32:
            break
    
    # save the image
    image_path = photoPath+'livecardimg.jpg'
    cv2.imwrite(image_path, frame)
              
    #release the webcam
    cap.release()
    cv2.destroyAllWindows()
    
    # return the file path to the image
    return image_path
# sn end
            
print('You are a master thief in the Middle Ages. You rob from the rich to survive, and because it is fun to deprive them of their wealth.')
print('What type of thief are you?')

#Character selection
charSelect = ''
charSuit = ''
print('Character types (what you are best at):')
print('1. Clubs - Brute strength')
print('2. Diamonds - Wit/Intelligence')
print('3. Hearts - Charm/Sociability')
print('4. Spades - Stealth/Dexterity')

while charSelect == '':
    charSelect = input('Enter the number next to the character you want to play: ')

    if charSelect == '1':
        charSuit = 'Clubs'
    elif charSelect == '2':
        charSuit = 'Diamonds'
    elif charSelect == '3':
        charSuit = 'Hearts'
    elif charSelect == '4':
        charSuit = 'Spades'
    else:
        print('Invalid choice, please type a number from 1-4')
        charSelect = ''

print(charSuit)

print('')
print("Scenario:")
print("You've heard of a dinner party being hosted by one of the wealthiest citizens, taking place a few nights from today.")

if charSuit == 'Diamonds' or 'Hearts':
    print("You spend the next few days asking your underground contacts. They describe who will be there and get you an invite under an assumed name.")
    if charSuit == 'Hearts':
        print("They also drop your name with the right people, so you will have an easier time being welcomed.")
        charm += 3
    else:
        print("They also give you a basic layout of the house from times they've been there, along with where a safe is located. You think there's a back way upstairs using this knowledge.")
        intellect += 3
else:
    print("You spend the next few days asking your underground contacts. They get you blueprints and advise that typically guards are roaming the house during parties.")
    if charSuit == 'Clubs':
        print("You also spend time studying the guards you can see roaming the estate, looking for weaknesses in their routines and their armor.")
        combat += 3
    if charSuit == 'Spades':
        print("You also spend time studying the routes the guards take and find some weaknesses in their rotations.")
        stealth += 3
        
print("The night of the party arrives.")
print('')
while True:
    if charSuit == 'Diamonds' or charSuit == 'Hearts':
        print('You journey inside, giving the guard at the gate your pass.')
        if charSuit == 'Hearts':
            print("Once inside, a group of people head your way, wanting to talk to you about your exploits fighting bandits, saving children, etc")
        else:
            print("Once inside, you start wandering around, getting your bearings, confirming certain details, and mingling with the guests to blend in.")

        print("After an appropriate amount of time, you are beginning to think about making your move on the safe. You know it is upstairs, but you are unsure if there are guards.")
        if charSuit == 'Hearts':
            print("Do you:")
            print("1. Storm out of the room, causing chaos in hopes no one follows")
            print("2. Try to slip up the stairs without being seen")
            choice = input("Type the number of your choice: ")
            if choice == '1':
                #Need a heart card for Intimidating Presence, end game if fail

                print("You need a card of suit " + abilityDict['Intimidating Presence']['Suit'])
                
                result = -1
                while result == -1:
                    cardChosen = imgSel()
                    result = abilityCheck(charSuit,'Intimidating Presence',7,cardChosen)
                
                if result == 0:
                    print("While trying to make a scene and cause people to be glad you are gone, you went too far and were forced to leave the party. You have failed.")
                    break
                else:
                    print("Your plan works, people are so distracted talking about how much of a boor you were, that you were able to slip away unnoticed.")
            else:
                #Need a heart card for Bluff when they get caught, end game if fail

                print("You need a card of suit " + abilityDict['Bluff']['Suit'])
                
                result = -1
                while result == -1:
                    cardChosen = imgSel()
                    result = abilityCheck(charSuit,'Bluff',7,cardChosen)
                
                if result == 0:
                    print("You attempt to sneak upstairs, but an admirer of your great works in the stories they've been told sees you trying to go upstairs and starts pestering you. Your narrow window has closed, and now dinner is served. You have failed.")
                    break
                else:
                    print("You attempt to sneak upstairs and are caught by a passerby. You quickly concoct a story about the host needing to see you upstairs in private, and they buy it.")
            path = 'staircase'
        else:
            print("Do you:")
            print("1. Go up the staircase")
            print("2. Find the room downstairs that may have a back way up")
            choice = input("Type the number of your choice: ")
            if choice == '1':
                #Need a hearts card for Bluff, end game if fail

                print("You need a card of suit " + abilityDict['Bluff']['Suit'])
                
                result = -1
                while result == -1:
                    cardChosen = imgSel()
                    result = abilityCheck(charSuit,'Bluff',7,cardChosen)
                
                if result == 0:
                    print("You attempt to sneak upstairs, but an admirer of your great works in the stories they've been told sees you trying to go upstairs and starts pestering you. Your narrow window has closed, and now dinner is served. You have failed.")
                    break
                else:
                    print("You attempt to sneak upstairs and are caught by a passerby. You quickly concoct a story about the host needing to see you upstairs in private, and they buy it.")
                path = 'staircase'
            else:
                #Need a diamonds card for King of Preparation, end game if fail

                print("You need a card of suit " + abilityDict['King of Preparation']['Suit'])
                
                result = -1
                while result == -1:
                    cardChosen = imgSel()
                    result = abilityCheck(charSuit,'King of Preparation',7,cardChosen)
                
                if result == 0:
                    print("You thought that you had the timing of the night's events down pat, but either they changed plans or you lost track of time. Either way, your path to the back room is now blocked, and you would be noticed going up the main staircase. You have failed.")
                    break
                else:
                    print("You slip past the servants without too much notice, and you find the dumbwaiter right where your contact said it would be. With a glance over your shoulder, you slip into the dumbwaiter and hoist yourself upstairs.")
                path = 'dumbwaiter'

        if path == 'staircase':    
            print("You make it upstairs to find a guard standing outside the room with the safe. Currently he is facing your way, but furniture is blocking his view of you.")
            if charSuit == "Hearts":
                #Need a Hearts for Lure, end game if fail

                print("You need a card of suit " + abilityDict['Lure']['Suit'])
                
                result = -1
                while result == -1:
                    cardChosen = imgSel()
                    result = abilityCheck(charSuit,'Lure',7,cardChosen)
                
                if result == 0:
                    print("You try to get the guard to leave their post, or let you by him to the bathroom you swear is back there, but he gets annoyed and tosses you downstairs. You have failed.")
                    break
                else:
                    print("The guard is alarmed at the story about one of the servants trying to poison the guests. He was in such a rush that he insisted you go downstairs, but didn't make sure you followed.")
            else:
                #Need a Hearts for Bluff, end game if fail

                print("You need a card of suit " + abilityDict['Bluff']['Suit'])
                
                result = -1
                while result == -1:
                    cardChosen = imgSel()
                    result = abilityCheck(charSuit,'Bluff',7,cardChosen)
                
                if result == 0:
                    print("You try to get the guard to leave their post, or let you by him to the bathroom you swear is back there, but he gets annoyed and tosses you downstairs. You have failed.")
                    break
                else:
                    print("The guard is alarmed at the story about one of the servants trying to poison the guests. He was in such a rush that he insisted you go downstairs, but didn't make sure you followed.")
        else:
            print("Your intel was correct, a dumbwaiter takes you to a section of the hallway upstairs. As you slowly slide open the hatch, a guard is stationed a few feet away, currently looking the other direction")
            #Need Spade for Vanish, end game if fail

            print("You need a card of suit " + abilityDict['Vanish']['Suit'])
                
            result = -1
            while result == -1:
                cardChosen = imgSel()
                result = abilityCheck(charSuit,'Vanish',7,cardChosen)

            if result == 0:
                print("You attempt to climb out of the dumbwaiter, but your foot catches on a nail you didn't previously see. As you tumble out, the noise catches the guard's attention, and you are unable to hide behind anything. You are ejected from the party and sent to jail. You have failed.")
                break
            else:
                print("As you slide out, you make a minor noise, but are able to hide behind furniture before the guard turns around. They look back for a moment, and come your way, but return to their post before getting too close.")
    else:
        print('')
        print("You make your way behind the estate until you find the area that provides the most cover")
        if charSuit == "Spades":
            print("Following the plan you set forth, you wait until the gap in the rotation happens, then you bolt over the wall and head for the building.")
            #Need Spade for Vanish as guard rounds corner, end game if fail

            print("You need a card of suit " + abilityDict['Vanish']['Suit'])
                
            result = -1
            while result == -1:
                cardChosen = imgSel()
                result = abilityCheck(charSuit,'Vanish',7,cardChosen)

            if result == 0:
                print("As you were sliding behind a column, you step on a twig which snaps. The guard spots you and yells for help. You are forced to flee. You fail.")
                break
            else:
                print("You slip behind a support of a balcony above, getting as close to the building as possible. The guard walks by, unaware of your presence.")
            #Need Spade for Excellent Finesse, end game if fail

            print("You need a card of suit " + abilityDict['Excellent Finesse']['Suit'])
                
            result = -1
            while result == -1:
                cardChosen = imgSel()
                result = abilityCheck(charSuit,'Excellent Finesse',7,cardChosen)

            if result == 0:
                print("You try to climb up to the balcony, but the rain this morning caused the side of the building to be slippery, and you proceed to tumble to the ground. You soon find yourself in jail. You fail.")
                break
            else:
                print("The guard is momentarilly distracted, which gives you time to scale the wall, quiet as a church mouse. You land on the balcony just as the guard turns around, staring at the place you were standing a moment ago.")
        else:
            print("You found a part of the rotation where only a single guard remained in an area for a while. After watching for a bit, you confirm the same routine is in effect tonight.")
            #Need Club for Brutal, end game if fail

            print("You need a card of suit " + abilityDict['Brutal']['Suit'])
                
            result = -1
            while result == -1:
                cardChosen = imgSel()
                result = abilityCheck(charSuit,'Brutal',7,cardChosen)

            if result == 0:
                print("You rush the lone guard, but you misjudged the height of the wall and your foot catches on the top. You fall at the feet of the guard, who yells for help. You fail.")
                break
            else:
                print("As the guard falls, a second guard is rounding the corner. Drats! That didn't happen the other night.")
            #Need Club for Knock-out King, end game if fail

            print("You need a card of suit " + abilityDict['Knock-out King']['Suit'])
                
            result = -1
            while result == -1:
                cardChosen = imgSel()
                result = abilityCheck(charSuit,'Knock-out King',7,cardChosen)

            if result == 0:
                print("You must not have seen all of the guards, as this one has a helmet on. You don't realize that until you try to rap him on the head. You fail.")
                break
            else:
                print("After taking out this guard quietly and hiding both behind a stack of firewood, you now feel confident in being able to scale the building in peace.")

        print("As you climb through the window, you know that you are in an adjacent room to where the safe resides.")

        if charSuit == "Spades":
            print("You know a guard stands, well, guard up here, so you quietly open the door.")
            #Need Spade for King of Preparation, end game if fail

            print("You need a card of suit " + abilityDict['King of Preparation']['Suit'])
                
            result = -1
            while result == -1:
                cardChosen = imgSel()
                result = abilityCheck(charSuit,'King of Preparation',7,cardChosen)

            if result == 0:
                print("You step into the hallway as carefully as you can, but you weren't prepared for all of the creaky boards. You fail as the guard spins and calls for help.")
                break
            else:
                print("You know that the distance is a bit too far, and the floor boards very creaky. Luckily, you brought a tranquilizer dart. A moment later, the guard is passed out.")
        else:
            print("You open the door quietly and see a guard standing a few feet away. You quietly approach, but the floorboards creak.")
            #Need Club for Brutal

            print("You need a card of suit " + abilityDict['Brutal']['Suit'])
                
            result = -1
            while result == -1:
                cardChosen = imgSel()
                result = abilityCheck(charSuit,'Brutal',7,cardChosen)

            if result == 0:
                print("As you open the door and try to rush the guard, don't see the chair sticking out from the darkness created by a hutch. You fail as you fall flat in front of the guard, who yells for help.")
                break
            else:
                print("You see him turning, so you rush forward, hitting him in the gut to knock the wind out, followed by a swift knock over the back of his head. He drops to the ground, with you guiding the fall to reduce the noise.")

    print('')
    print("The room with the safe is locked, but you think to check the guard for a key. Amateurs, the guard had one on him.")
    print("The safe is in a closet, hidden behind some clothes. The combination you were given works, and a moment later you have a bag of precious gems, coins, and trinkets.")
    print("The bag is too noisy to even think about going out through the ground floor. You will need to jump.")

    if charSuit == "Hearts" or "Clubs":
        #Need Clubs for Strong Recovery, end game if fail

        print("You need a card of suit " + abilityDict['Strong Recovery']['Suit'])
                
        result = -1
        while result == -1:
            cardChosen = imgSel()
            result = abilityCheck(charSuit,'Strong Recovery',7,cardChosen)

        if result == 0:
            print("You land hard and let out a yell. Even though it's dark in the yard, the guards quickly find you before you can get up. You fail.")
            break
    elif charSuit == "Diamonds":
        #Need Diamonds for King of Preparation, end game if fail

        print("You need a card of suit " + abilityDict['King of Preparation']['Suit'])
                
        result = -1
        while result == -1:
            cardChosen = imgSel()
            result = abilityCheck(charSuit,'King of Preparation',7,cardChosen)

        if result == 0:
            print("You had planned for a bundle of hay to be below the window. The person promised they would have it there, and they were paid handsomely for this. So you didn't look before jumping. You know you broke bones when you landed, but that only lasted a moment before going unconscious. When you wake, you are in pain in a jail cell. You fail.")
            break
    else:
        #Need Spades for Excellent Finesse, end game if fail

        print("You need a card of suit " + abilityDict['Excellent Finesse']['Suit'])
                
        result = -1
        while result == -1:
            cardChosen = imgSel()
            result = abilityCheck(charSuit,'Excellent Finesse',7,cardChosen)

        if result == 0:
            print("You land hard and let out a yell. Even though it's dark in the yard, the guards quickly find you before you can get up. You fail.")
            break
    
    print('')
    print("You land safely, causing a minimal amount of noise. No guards appear, so you quickly dive over the back wall and head for the neighboring forest.")
    print("After travelling a sufficient distance, you take a moment to catch your breath, assemble your thoughts, and go back to town.")
    print("The next few days will be spent talking to fences, eating well, and planning your next heist.")
    print("Congratulations")
    break